In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.neighbors import NearestNeighbors
import scipy


In [2]:
reviews_hotels = pd.read_csv("d:/Henry/Proyecto Final/Dataset_Consolidado/reviews_hotel_para_Eda.csv",index_col=0)
sitios_hotels = pd.read_csv("d:/Henry/Proyecto Final/Dataset_Consolidado/sitios_hotel_para_Eda.csv",index_col=0)
reviews_sentimiento = pd.read_csv("D:/Henry/Proyecto Final/Dataset_models/reviews_sentiment_sbcBI_2.csv")

In [3]:
sitios_hotels[pd.isnull(sitios_hotels["cat_name"])]

,name_hotel,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,...,t.child_care,t.familiar/group,t.all_payment_method,t.lgtbq_friendly,t.outdoor_seating,t.breakfast,t.delivery,t.live_performance,t.covid,t.fireplace
12931,NaN,"NA, 9907 FREDERICKSBURG RD, SAN ANTONIO, TX 78240",0x865c675320103b85:0xc24ffdb35245e49d,NaN,29.533223,-98.5771,['resort hotel'],4.8,16,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
# en cada dataset vamos a pasar las columnas de interes
    # hacer un groupby en tablas reviews para hallar el sentimiento esperado y numero de comentarios por local
    # pasamos de reviews a sitios
    # de sitios pasamos directas las columnas a reviews

# filtramos las columnas a usar solamente
# tiramos las columnas a dummies
# realizamos el modelo de recomendacion


## Modelo desde Sitios

In [4]:
reviews_sentimiento["sentimiento"] = reviews_sentimiento.apply(lambda row: 1 if row.sb_pos>row.sb_neu and row.sb_pos>row.sb_neg else (-1 if row.sb_neg>row.sb_pos and row.sb_neg>row.sb_neu else 0 ),axis=1)
reviews_sentimiento_prom = reviews_sentimiento.groupby("gmap_id").mean()
sitios_hotels = sitios_hotels.merge(reviews_sentimiento_prom,on="gmap_id",how="left")
sitios_hotels["sentimiento"] = sitios_hotels["sentimiento"].fillna(0)

C:\Users\wongp\AppData\Local\Temp\ipykernel_9496\865838958.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  reviews_sentimiento_prom = reviews_sentimiento.groupby("gmap_id").mean()


In [5]:
sitios = sitios_hotels[["avg_rating","num_of_reviews","state_name","cat_name","sentimiento"]]

In [6]:
sitios

,avg_rating,num_of_reviews,state_name,cat_name,sentimiento
0,5.0,8,Washington,Hotel,1.000000
1,5.0,1,California,Lodge,0.000000
2,3.9,8,Ohio,Inn,0.000000
3,3.6,8,North Carolina,Lodge,-0.250000
4,4.3,8,Texas,Lodge,0.666667
...,...,...,...,...,...
15703,4.5,6,Indiana,Lodge,0.000000
15704,4.4,7,Delaware,Inn,0.000000
15705,4.3,58,New Mexico,Lodge,0.000000
15706,5.0,3,Idaho,Lodge,0.000000


In [7]:
from sklearn.preprocessing import OneHotEncoder

ohe_state = OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe_state.fit(sitios[["state_name"]])

ohe_cat = OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe_cat.fit(sitios[["cat_name"]])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [8]:
dummies_cat = ohe_cat.transform(sitios[["cat_name"]])
headers_cat = np.array(ohe_cat.categories_).tolist()[0]
df_dummies_cat = pd.DataFrame(dummies_cat,columns=headers_cat)
sitios = pd.concat([sitios,df_dummies_cat],axis=1)
sitios.drop("cat_name",axis=1,inplace=True)

dummies_state = ohe_state.transform(sitios[["state_name"]])
headers_state = np.array(ohe_state.categories_).tolist()[0]
df_dummies_state = pd.DataFrame(dummies_state,columns=headers_state)
sitios = pd.concat([sitios,df_dummies_state],axis=1)
sitios.drop("state_name",axis=1,inplace=True)

sitios

,avg_rating,num_of_reviews,sentimiento,Hotel,Inn,Lodge,Motel,Resort,NaN,Alabama,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,"Washington, D.C.",West Virginia,Wisconsin,Wyoming
0,5.0,8,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,5.0,1,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.9,8,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.6,8,-0.250000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.3,8,0.666667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15703,4.5,6,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15704,4.4,7,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15705,4.3,58,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15706,5.0,3,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
sitios_sparse = scipy.sparse.csr_matrix(sitios)

from sklearn.neighbors import NearestNeighbors
n_neighbors=6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(sitios_sparse)

In [16]:
#parametros de entrada dinamicos para streamlit

def get_sparse_evaluardo(v_avg_rating,v_num_of_reviews,v_state_name,v_cat_name,v_sentimiento):
    #consilidamos parametros en df
    evaluar_recomend = pd.DataFrame({
                    "avg_rating":v_avg_rating,
                    "num_of_reviews":v_num_of_reviews,
                    "state_name":v_state_name,
                    "cat_name":v_cat_name,
                    "sentimiento":v_sentimiento
                    },index=[0])

    #creamos las columnas dummies para poder ingresar al modelo
    encode_state = ohe_state.transform(pd.DataFrame({"state_name":v_state_name},index=[0]))
    df_state_dummies_v = pd.DataFrame(encode_state,columns=headers_state)

    encode_cat = ohe_cat.transform(pd.DataFrame({"cat_name":v_cat_name},index=[0]))
    df_cat_dummies_v = pd.DataFrame(encode_cat,columns=headers_cat)

    evaluar_recomend = pd.concat([evaluar_recomend,df_cat_dummies_v,df_state_dummies_v],axis=1)
    
    evaluar_recomend.drop(["cat_name","state_name"],axis=1,inplace=True)

    evaluar_recomend_sparse = scipy.sparse.csr_matrix(evaluar_recomend)

    return evaluar_recomend_sparse

evaluador = get_sparse_evaluardo(4.0,10,"Washingto","Hotel",1.0)


In [17]:
evaluador

<1x61 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [18]:
dif, ind = nneighbors.kneighbors(evaluador)

print("Recomendaciones")
print("="*80)
sitios_hotels.loc[ind[0][:], :]

Recomendaciones


,name_hotel,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,...,t.covid,t.fireplace,user_id,time,rating,long_text,sb_pos,sb_neu,sb_neg,sentimiento
12359,Olympic Dreams Chalet,"OLYMPIC DREAMS CHALET, 66 FOX FARM RD, WILMING...",0x4ccaf95cbaf2fdef:0x5b8aa703a86830de,NaN,44.362676,-73.836796,"['vacation home rental agency', 'hotel']",4.9,13,NaN,...,0.0,0.0,1.112289e+20,1.544686e+12,5.000,136.000,0.956233,0.022833,0.020934,1.00
11281,Lancaster County Sportsman's,"LANCASTER COUNTY SPORTSMAN'S, 573 HILLDALE RD,...",0x89c62db04e6e8f43:0x529fcf9226fba10c,NaN,39.872580,-76.308308,"['roller skating rink', 'hotel']",4.7,13,NaN,...,0.0,0.0,1.083498e+20,1.587497e+12,5.000,87.500,0.942531,0.029824,0.027645,1.00
10339,Romancing the stone,"ROMANCING THE STONE, 5178 TOWNSHIP RD 356, MIL...",0x883713b41aeda293:0xe4f70bab8a349144,NaN,40.550929,-81.805004,['hotel'],5.0,13,NaN,...,0.0,0.0,1.082698e+20,1.554215e+12,5.000,149.750,0.782303,0.041214,0.176483,0.75
12187,Papa's Place,"PAPA'S PLACE, MCKEE RD, FRAZEYSBURG, OH 43822",0x88382988f19008d9:0x462615018c065678,NaN,40.275756,-82.247089,['hotel'],4.8,15,NaN,...,0.0,0.0,1.062498e+20,1.570680e+12,4.875,184.125,0.876861,0.045737,0.077402,1.00
1689,Jordan Welcome Center,"JORDAN WELCOME CENTER, 6714 OK-77D, DAVIS, OK ...",0x87b2d639d8af3fd9:0x87bc5746b9bd31cc,NaN,34.423812,-97.110629,['hotel'],4.8,15,NaN,...,0.0,0.0,1.098755e+20,1.503601e+12,4.800,127.000,0.892339,0.041441,0.066220,1.00
5273,Lake Anna,"LAKE ANNA, 6600 ZACHARY TAYLOR HWY, MINERAL, V...",0x89b6aa2fe2b5bbdf:0xb428bf1081eb624e,NaN,38.099021,-77.893160,['hotel'],4.7,15,NaN,...,0.0,0.0,1.135827e+20,1.546377e+12,4.600,39.600,0.939319,0.034284,0.026397,1.00
